In [29]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter
import networkx as nx
import math


# Ucitavanje grafova

In [30]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph

def loadGraph(input_file):
    global d
    G = nx.Graph()
    for j in range(0,n):
        G.add_node(j)

    f = open(input_file, "r")
    string = f.readline()
    string = f.readline()
    string = f.readline()
    for i in range(0, m):
        string = f.readline()
        string = string.split()
        j = int(string[0])-1
        k = int(string[1])-1
        G.add_edge(j, k)
    f.close()
    d = []
    j = 1
    for i in range(n):
        d.append([float("inf")]*(j))
        j+=1
    for i in range(G.number_of_nodes()):
        ds = nx.single_source_shortest_path_length(G, i)
        for key in ds:
            if i >= key:
                d[i][key] = ds[key]
            else:
                d[key][i] = ds[key]
    print("n: " + str(G.number_of_nodes()))
    print("m: " + str(G.number_of_edges()))
    print("ncc: " + str(nx.number_connected_components(G)))


# Pomocne funkcije

In [31]:
def all_pairs_distance_matrix(graph):
  
    nodes = sorted(graph.keys()) 
    n = len(nodes)
    distance_matrix = [[math.inf] * n for _ in range(n)]  

    for start in nodes:
        visited = {node: False for node in nodes}
        d_local = {node: math.inf for node in nodes}
        q = deque()
        d_local[start] = 0
        visited[start] = True
        q.append(start)

        while q:
            current = q.popleft()
            for neighbor in graph.get(current, []):
                if not visited[neighbor]:
                    visited[neighbor] = True
                    q.append(neighbor)
                    d_local[neighbor] = d_local[current] + 1

       
        for node in nodes:
            distance_matrix[start - 1][node - 1] = d_local[node]

    return distance_matrix

def valid_burning_sequence(s, n, d):
  
    counter = 0

    for i in range(n):
        b = len(s)
        for j in range(b):
            if d[i][s[j] - 1] <= b - (j + 1): 
                counter += 1
                break

    return counter == n

# Generisanje nasumicnih burning sekvenci

In [32]:
def bfs_shortest_paths(graph, start):
    dist = {node: float('inf') for node in graph} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graph}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graph[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_neighbors(graph):
    neighbors_count = {node: len(graph[node]) for node in graph}
    return neighbors_count

def choose_initial_node(graph, neighbors_count):
    #kombinuje heuristicki i nasumicni izbor pocetnog cvora
    
    top_k = int(len(graph)/3)
    sorted_nodes = sorted(graph.keys(), key=lambda node: neighbors_count[node], reverse=True)
    
    # uzmi top_k cvorova (ili manje ako ih nema dovoljno)
    top_candidates = sorted_nodes[:top_k]
    
    # nasumicno izaberi jednog od kandidata
    return random.choice(top_candidates)

In [33]:
def greedy_burning(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    
    neighbors_count = calculate_neighbors(graph)

    current_node = choose_initial_node(graph, neighbors_count)
    
    
        
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
   
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        
        
        if NB:
            weights = []
            for node in NB:
                if node not in FB2:
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    weights.append(notburned_neighbors + 1)  # dodajemo 1 da ne bismo imali težinu 0
                    
           
            # normalizacija tezina
            total_weight = sum(weights)
            probabilities = [w / total_weight for w in weights]
            
            candidates = []
            for node in NB:
                if node not in FB2:
                    candidates.append(node)
           
            if candidates:
                best_node = random.choices(list(candidates), weights=probabilities, k=1)[0]
            else:
                best_node = None

                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
           
        elif FB2 and not NB:
              S.append(random_node(FB2))
              break
        else:
            
            break
           
    return S
def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    sequences = []
    for _ in range(num_sequences):
        sequence = greedy_burning(graph)
        if sequence not in sequences and valid_burning_sequence(sequence, len(graph), all_pairs_distance_matrix(graph)):
        #if sequence not in sequences:
            sequences.append(sequence)
    return sequences


In [34]:
def transpose_list_of_lists(input_list):
    # Transponuje listu listi tako da prvi elementi svih podlisti budu u prvoj listi, drugi u drugoj, itd.
    
    transposed = []
    for item in zip(*input_list):
        unique_item_list = list(set(item))
        transposed.append(unique_item_list)
    return transposed

#burning_sequences_t = transpose_list_of_lists(burning_sequences)
#print(burning_sequences_t)


# PuLP

In [ ]:
    
def ILP(L,k):
    global d  
    
    #model
    model = LpProblem("ILP_COV", LpMinimize)
    
    # definisemo varijable
    b = [LpVariable("b_{}".format(j+1), cat='Binary') for j in range(n)]
    x = [[LpVariable("x_{}_{}".format(r+1, i+1), cat='Binary') for i in range(n+1)] for r in range(k)]
    
    # ciljna funkcija
    s1= 0
    for i in range(k):
        for j in range(n):
            s1 += x[i][j]
    model+=s1
        
    
    
    #domensko ogranicenje
    for i in range(int(k/2)):
        #print(len(burning_sequences_t[i]))
        model += lpSum(x[i][j] for j in burning_sequences_t[i]) == 1

    
    #ogranicenja
    #(13)
    for i in range(k):
        suma=0
        for j in range(n):
            suma+=x[i][j]
        model +=(suma<=1)
        
    #(14)
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        model +=(b[j]<=suma)
        #(12)
    for i in range(k):
        sum2 = 0
        if i == 0:
            sum1 = 1
        else:
            sum1 = 0
        for j in range(n):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        model +=(sum2<=sum1)

    
    #(15)
    model +=lpSum(b[j] for j in range(n)) == n 
    
    
    
    
   # Rjesavanje modela
    #model.solve(PULP_CBC_CMD(msg=True, timeLimit=600, options=["gap 0.3"]))

    model.solve(PULP_CBC_CMD(mip=True, msg=True, timeLimit=60, gapRel=0.2))

    #model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0.05))
    
    if LpStatus[model.status] in ['Optimal', 'Suboptimal', 'Feasible']:
        print(f"Solution Found (Status: {LpStatus[model.status]}): Objective Value = {model.objective.value()}")
        for i in range(k):
            for j in range(n):
                if x[i][j].value() == 1:
                    print(f"x[{i}, {j}] = 1")
    else:
        print("No feasible solution found within the time limit.")
        print(model.status)
        print("Solver status unclear:", LpStatus[model.status])
        #print(model)
    if model.status not in [LpStatusOptimal, LpStatusNotSolved, LpStatusInfeasible, LpStatusUnbounded, LpStatusUndefined]:
        print("Solver status unclear:", LpStatus[model.status])


    s = []
    
    x_out = [[0]*n for i in range(k)]
    for r in range(k):
        i = 0
        for e in x_out[r]:
            if e == 1:
                break
            i += 1
 
    s.reverse()
   

    return s
        
def main(ni, mi, input_file, Li, Ui):
    n = ni
    m = mi
    U = Ui
    L = Li
    loadGraph(input_file)
    s = ILP(L,U)

if __name__ == "__main__":
    bs = []
    n = 0
    m = 0
    d = {}
    bs_size = float("inf")
    folder_dataset = 'grafovi/'
    dataset = [
        ['primjer.mtx',17,19,2,5],
        #['karate.mtx',34,78,2,3], 
        #['chesapeake.mtx',39,170,1,3], # instance, n, m, l, h
        #['dolphins.mtx',62,159,2,4],
        #['rt-retweet.mtx',96,117,2,5],
        #['polbooks.mtx',105,441,2,4],
        #['adjnoun.mtx',112,425,2,4],
        #['ia-infect-hyper.mtx',113,2196,1,3],
        #['C125-9.mtx',125,6963,1,3],
        #['ia-enron-only.mtx',143,623,2,4],
        #['c-fat200-1.mtx',200,1534,3,7],
        #['c-fat200-2.mtx',200,3235,2,5],
        #['c-fat200-5.mtx',200,8473,1,3],
        #['sphere.mtx',258,1026,3,7],
        #['DD244.mtx',291,822,4,7],
        #['ca-netscience.mtx',379,914,3,6],
        #['infect-dublin.mtx',410,2765,2,5],
        ['bio-diseasome.mtx',516,1188,5,7],
        #['web-polblogs.mtx',643,2280,3,5],
        #['DD687.mtx',725,2600,4,8],
        #['rt-twitter-copen.mtx',761,1029,3,7],
        #['DD68.mtx',775,2093,5,9],
        #['ia-crime-moreno.mtx',829,1475,3,7],
        #['DD199.mtx',841,1902,6,12],
        #['soc-wiki-Vote.mtx',889,2914,3,6],
        #['DD349.mtx',897,2087,6,12],
        #['DD497.mtx',903,2453,6,11],
        #['socfb-Reed98.mtx',962,18812,2,4],
        #['delaunay_n10.mtx',1024,3056,4,9],
        #['email-univ.mtx',1133,5451,2,5],
        #['econ-mahindas.mtx',1258,7513,2,5],
        #['ia-fb-messages.mtx',1266,6451,2,5],
        #['bio-yeast.mtx',1458,1948,4,9],
        #['tech-routers-rf.mtx',2113,6632,3,6],
        #['facebook.mtx',4039,88234,2,4],
        #['squirrel.mtx',5201,198493,2,6],
        #['grid80x80.mtx',6400,12640,11,24],
       
        ]
    for i in range(len(dataset)):
        print("________________________________________________")
        instance = dataset[i][0]
        print("instance: " + instance)
        filename = f"grafovi/{instance}"
        #filename = f"grafovi/dolphins.mtx"
        
        graph = load_graph_from_mtx(filename)
        j = 0
        
        burning_sequences = generate_multiple_random_burning_sequences_with_greedy(graph, 50)
        
        burning_sequences_t = transpose_list_of_lists(burning_sequences)
        for seq in burning_sequences:
            print(seq)
        
        n = dataset[i][1]
        m = dataset[i][2]
        L = dataset[i][3]
        U = dataset[i][4]
        startTime = time.time()
        main(n, m, folder_dataset + dataset[i][0], L, U)
        print("--- %s sekundi ---" % (time.time() - startTime))
            
    
    

________________________________________________
instance: primjer.mtx
[1, 13, 15, 14]
[2, 8, 15, 17, 13]
[10, 8, 15, 9]
[1, 14, 15, 16]
[1, 11, 17, 14]
[4, 13, 15, 16]
[3, 16, 13, 8]
[2, 12, 17, 11]
[3, 13, 16, 8]
[1, 15, 17, 14]
[4, 10, 11, 13]
[4, 10, 13, 15, 11]
[4, 13, 15, 11]
[1, 14, 16, 12]
[1, 11, 17, 13]
[2, 14, 16, 11]
[4, 17, 15, 16]
[1, 16, 17, 14]
[4, 13, 11, 16]
[4, 11, 13, 14]
[2, 16, 13, 14]
[1, 11, 17, 16]
[4, 16, 17, 9]
[3, 15, 13, 17]
[2, 14, 15, 12]
[3, 13, 16, 15]
[1, 16, 17, 11]
[4, 15, 17, 13]
[2, 16, 13, 7]
[3, 12, 13, 9]
[3, 12, 13, 15]
[3, 12, 13, 17]
[2, 13, 11, 16, 12]
[4, 15, 17, 7]
[2, 14, 12, 15, 11]
[10, 11, 8, 9]
[3, 9, 12, 16]
[1, 16, 17, 13]
[10, 16, 9, 5]
[10, 15, 9, 8]
[4, 11, 17, 14]
[10, 12, 9, 5]
[1, 17, 15, 12]
[10, 12, 9, 15]
[4, 7, 17, 16]
[2, 17, 15, 12]
n: 17
m: 19
ncc: 1
Solution Found (Status: Optimal): Objective Value = 4.0
x[0, 1] = 1
x[1, 14] = 1
x[2, 13] = 1
x[3, 3] = 1
--- 0.08971476554870605 sekundi ---
______________________________